# 프로세스 준비

In [ ]:
## 설치 필요 라이브러리 목록
# !pip install ee
# !pip install geemap
# !pip install ipympl

In [1]:
import ee, geemap, os, shutil
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import ipyleaflet
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import numpy as np
import cufflinks as cf
cf.go_offline(connected=True)

In [2]:
geemap.ee_initialize()

In [3]:
def addNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands([ndvi])

In [4]:
def maskS2clouds(image):
    qa = image.select('QA60')
    
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    
    mask = (qa.bitwiseAnd(cloudBitMask).eq(0) and qa.bitwiseAnd(cirrusBitMask).eq(0))
        
    return image.updateMask(mask)

In [5]:
def clipPoint(image):
    return image.clip(ee.Geometry.Point(point).buffer(250).bounds())\
                .reduceRegions(
                     collection = points,
                     reducer = ee.Reducer.mean(),
                     scale = 10)

---
# 데이터 준비

In [6]:
table = geemap.shp_to_ee('shp file here')
df = geemap.ee_to_df(table)

In [7]:
coords = df[['Location', 'Lon', 'Lat']].drop_duplicates().copy()

coords_dict = { }

## 데이터가 있는 Location만 진행하는 경우

In [ ]:
loc_list = list(map(lambda x: x.strip('.xlsx'), os.listdir('data')))
df = df[df['Location'].isin(loc_list)].sort_values(['Location', 'BUFF'])

loc_list

## 모든 Location 대상으로 진행하는 경우

In [ ]:
loc_list = list(coords['Location'].values)
loc_list

---

# 위성 이미지 출력

- location 변수에 Location 코드 입력하여 사용

In [9]:
location = ''

point = coords_dict[f'{location}_coord']

start_date = '2022-01-01'
end_date = '2022-12-01'

images = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
       .select(['B2', 'B3', 'B4', 'B8', 'QA60'])
       .filterDate(start_date, end_date)
       .filterBounds(ee.Geometry.Point(point))
       .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)
       .map(addNDVI)
       .map(maskS2clouds))

vis = {'bands': ['B4', 'B3', 'B2'],
       'min': 0,
       'max': 3000,
       'gamma':1.4  }

In [ ]:
''' 이미지컬렉션 사이즈 확인 '''
size = images.size()
print(f'{location} data size : {size.getInfo()}')

images_list = images.toList(size)
print(f'{location} data_list size : {images_list.length().getInfo()}')

In [11]:
Map = geemap.Map()
Map.setCenter(point[0], point[1], 16)
Map.addLayer(ee.Image(images_list.get(2)).clip(ee.Geometry.Point(point).buffer(500).bounds()), vis, 'Sentinel-2')
Map.addLayer(ee.Image().paint(ee.Geometry.Point(point).buffer(250).bounds(), 0, 1), {'palette': 'FF0000'}, 'Box Outline')
Map.addLayer(ee.Image().paint(ee.Geometry.Point(point).buffer(30), 0, 1), {'palette': 'FFFF00'}, 'Center point')
Map

Map(center=[33.3179, 126.568], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

---
# 데이터 생성 및 전처리

## 위성 이미지 png 저장

In [13]:
''' 일자 및 폴더 지정 '''
if not os.path.exists('Sentinel-2 data'):
    os.makedirs('Sentinel-2 data')
os.chdir('Sentinel-2 data')
    
start_date = '2022-01-01'
end_date = '2022-12-01'
img_save_dir = f'{start_date}-{end_date}_Images'

In [ ]:
''' 폴더 생성 '''
# 해당 경로 폴더가 이미 있으면 삭제하고 진행하므로 img_save_dir 변수 확인하고 진행
if os.path.exists(img_save_dir):
    shutil.rmtree(img_save_dir)
os.makedirs(img_save_dir)

''' 전체 관측일자 Image 저장 '''
for location in loc_list:
    point = coords_dict[f'{location}_coord']

    images = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
           .select(['B2', 'B3', 'B4', 'B8', 'QA60'])
           .filterDate(start_date, end_date)
           .filterBounds(ee.Geometry.Point(point))
           .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)
           .map(addNDVI)
           .map(maskS2clouds))
    
    vis = {'bands': ['B4', 'B3', 'B2'],
       'min': 0,
       'max': 3000,
       'gamma':1.4  }
    
    size = images.size()
    images_list = images.toList(size)
    
    for num in range(size.getInfo()):
        targetImage = ee.Image(images_list.get(num)).clip(ee.Geometry.Point(point).buffer(300).bounds())
        datetime = ee.Date(ee.Image(images_list.get(num)).get('system:time_start')).format().getInfo()
        filename = f"{img_save_dir}\\{location}_Images\\{location}_{datetime[:10]}.png"
        
        for i in range(1, 5):
            if not os.path.exists(filename):
                filename = filename
            elif os.path.exists(filename):
                filename = f"{img_save_dir}\\{location}_Images\\{location}_{datetime[:10]}({i}).png"
            
        geemap.get_image_thumbnail(targetImage, filename, vis, dimensions=1000)
        print(f"{filename} 생성 완료.")
    print(f"총 {size.getInfo()}개 Image 생성 완료.", "="*70, sep="\n")
    print()

## 시계열 테이블 전처리 및 csv 저장

In [15]:
''' FeatureCollection 생성 '''
points = ee.FeatureCollection(table)

In [16]:
''' 일자 및 폴더 지정 '''
start_date = '2022-01-01'
end_date = '2022-12-01'
csv_save_dir = f'{start_date}-{end_date}_Time-Series'

In [ ]:
''' 폴더 생성 '''
# 해당 경로 폴더가 이미 있으면 삭제하고 진행하므로 csv_save_dir 변수 확인하고 진행
if os.path.exists(csv_save_dir):
    shutil.rmtree(csv_save_dir)  
os.makedirs(csv_save_dir)

''' 지점별로 시계열 csv파일 생성, 전처리, 이후 활용할 df 생성 반복 '''
for location in loc_list:
    point = coords_dict[f'{location}_coord']
    
    images = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
           .select(['B2', 'B3', 'B4', 'B8', 'QA60'])
           .filterDate(start_date, end_date)
           .filterBounds(ee.Geometry.Point(point))
           .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)
           .map(addNDVI)
           .map(maskS2clouds))

    features = images.map(clipPoint).flatten()
    
    geemap.ee_to_csv(features, f'{csv_save_dir}\\temp.csv')
    feature_df = pd.read_csv(f'{csv_save_dir}\\temp.csv', encoding='utf-8')
    feature_df_cleaned = feature_df.reset_index().sort_values(by=['Location', 'index'])
    feature_df_cleaned = feature_df_cleaned[['system:index', 'Location', 'BUFF_DIST', 'B2', 'B3', 'B4', 'B8', 'NDVI']].dropna() 
    feature_df_cleaned.rename(columns={'B2':'Blue', 'B3':'Green', 'B4':'Red', 'B8':'NIR'}, inplace=True)
    feature_df_cleaned[['Blue', 'Green', 'Red', 'NIR']] *= 0.0001
    feature_df_cleaned.rename(columns={'system:index':'DATE'}, inplace=True)
    feature_df_cleaned['DATE'] = feature_df_cleaned['DATE'].apply(lambda x: f"{x[:4]}-{x[4:6]}-{x[6:8]}")

    feature_df_cleaned.to_csv(f'{csv_save_dir}\\{location}_S2.csv', encoding='utf-8', index=False)
    globals()[f'{location}_df'] = feature_df_cleaned.copy()
    
os.remove(f'{csv_save_dir}\\temp.csv')

In [ ]:
''' 통합 df 생성 '''
total_df = pd.concat([globals()[f'{location}_df'] for location in loc_list], axis=0)
total_df.to_csv(f'{csv_save_dir}\\TOTAL_S2.csv', encoding='utf-8', index=False)

total_df.info()

---
# 데이터 시각화

- 데이터 로드

In [21]:
total_df = pd.read_csv(f'{csv_save_dir}\\TOTAL_S2.csv', encoding='utf-8')

## 지역별 관측 개수 시각화

In [15]:
# 그래프에 한글 출력 시 사용

font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

In [ ]:
counts = total_df['Location'].value_counts()/6

plt.figure(figsize=(8, 8))
plt.title('Observation number by Location')
bars = plt.bar(counts.index, counts, width=0.4)
plt.ylim(0, counts.max()*1.25) 

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2.0, 
             height, f'{height:.1f}', ha='center', va='bottom', size = 11)
    
plt.show()

## 일별 테이블

In [23]:
location = widgets.Dropdown(options=loc_list)
date = widgets.Dropdown(options=total_df['DATE'][total_df['Location'] == location.value].drop_duplicates())

def change_loc(*args):
    date.options = total_df['DATE'][total_df['Location'] == location.value].drop_duplicates()

location.observe(change_loc, 'value')

@interact
def showByDate( Location = location,
                Date = date ):

    return total_df.loc[(total_df.Location == Location) & (total_df.DATE == Date)]

interactive(children=(Dropdown(description='Location', options=('AMD', 'GCK', 'HAWS1', 'JJ', 'PYC'), value='AM…

## 일 / 밴드별 그래프

- 한 날짜에 관측 건수가 여러 건이 있어 중복되는 경우 평균으로 출력

In [24]:
location2 = widgets.Dropdown(options=loc_list)
date2 = widgets.Dropdown(options=total_df['DATE'][total_df['Location'] == location2.value].drop_duplicates())

def change_loc2(*args):
    date2.options = total_df['DATE'][total_df['Location'] == location2.value].drop_duplicates()

location2.observe(change_loc2, 'value')

@interact
def showByDateAndBand(Location = location2,
                      Date = date2,
                      Band = total_df.columns[-5:]
                     ):
    
    colorDict = {
        'Blue' : 'blue', 
       'Green' : 'green',
          'Red': 'red',
          'NIR': 'magenta',
         'NDVI': 'skyblue'
    }
    
    base = total_df[total_df['Location'] == Location].set_index(['BUFF_DIST'])
    data = base[base['DATE'] == Date].loc[:, Band].groupby('BUFF_DIST').mean()
    plt.figure(figsize=(9, 7))
    plt.plot(data, color=colorDict[Band])
    plt.xlabel('Buffer')
    plt.ylabel('Value', rotation=0, labelpad=14)
    plt.ylim(data.min() * 0.975, data.max() * 1.025)
    plt.title(f'{Location} | {Date} | {Band}')
    plt.grid(axis='y')
    plt.show()
    
    for buff, value in zip(data.index, data.values):
        print(f'{buff}M : {value:.7f}')

interactive(children=(Dropdown(description='Location', options=('AMD', 'GCK', 'HAWS1', 'JJ', 'PYC'), value='AM…

## 전체 기간 지역별 평균 그래프

In [25]:
location3 = widgets.Dropdown(options=loc_list)

@interact
def showByLocationMean(Location = location3):
    
    location_df = total_df[total_df['Location'] == Location]
    groupMean = location_df.drop(axis=1, columns=['Location' ,'BUFF_DIST']).groupby('DATE').mean()
    
    colorDict = {
            'Blue' : 'blue', 
           'Green' : 'green',
              'Red': 'red',
              'NIR': 'magenta',
             'NDVI': 'skyblue'
        }

    groupMean.iplot(kind='line', subplots=True, subplot_titles=True,
                    title='Timeseries graph by Bands (Mean)', 
                    vertical_spacing=0.1, dimensions=(1150, 800),
                    colors=colorDict)

interactive(children=(Dropdown(description='Location', options=('AMD', 'GCK', 'HAWS1', 'JJ', 'PYC'), value='AM…